In [1]:
import openeo
from openeo.rest.datacube import THIS

In [2]:
eodc_backend = "https://openeo.eodc.eu/openeo/1.1.0/"

In [4]:
conn = openeo.connect(eodc_backend).authenticate_oidc(provider_id="egi")

Authenticated using refresh token.


In [5]:
conn.describe_process("sen2like")

{'id': 'sen2like',
 'summary': 'Run the Sen2like processor ',
 'description': 'The Sen2like processor generates Sentinel-2-like harmonised or fused surface reflectances with higher periodicity by integrating additional compatible optical mission sensors. The source code is available at https://github.com/senbox-org/sen2like.',
 'categories': ['cubes'],
 'parameters': [{'name': 'data',
   'description': 'The datacube for which to generate L2F & L2H level data.',
   'optional': True,
   'schema': {'type': 'object', 'subtype': 'raster-cube'}},
  {'name': 'spatial_extent',
   'description': 'Limits the data to load from the collection to the specified bounding box or polygons.\n\nThe process puts a pixel into the data cube if the point at the pixel center intersects with the bounding box or any of the polygons (as defined in the Simple Features standard by the OGC).\n\nThe GeoJSON can be one of the following feature types:\n\n* A `Polygon` or `MultiPolygon` geometry,\n* a `Feature` with a `Polygon` or `MultiPolygon` geometry,\n* a `FeatureCollection` containing at least one `Feature` with `Polygon` or `MultiPolygon` geometries, or\n* a `GeometryCollection` containing `Polygon` or `MultiPolygon` geometries. To maximize interoperability, `GeometryCollection` should be avoided in favour of one of the alternatives above.\n\nSet this parameter to `null` to set no limit for the spatial extent. Be careful with this when loading large datasets! It is recommended to use this parameter instead of using ``filter_bbox()`` or ``filter_spatial()`` directly after loading unbounded data.',
   'optional': True,
   'schema': [{'type': 'object',
     'subtype': 'bounding-box',
     'required': ['west', 'south', 'east', 'north'],
     'properties': {'west': {'description': 'West (lower left corner, coordinate axis 1).',
       'type': 'number'},
      'south': {'description': 'South (lower left corner, coordinate axis 2).',
       'type': 'number'},
      'east': {'description': 'East (upper right corner, coordinate axis 1).',
       'type': 'number'},
      'north': {'description': 'North (upper right corner, coordinate axis 2).',
       'type': 'number'},
      'base': {'description': 'Base (optional, lower left corner, coordinate axis 3).',
       'type': ['number', 'null']},
      'height': {'description': 'Height (optional, upper right corner, coordinate axis 3).',
       'type': ['number', 'null']},
      'crs': {'description': 'Coordinate reference system of the extent, specified as as [EPSG code](http://www.epsg-registry.org/), [WKT2 (ISO 19162) string](http://docs.opengeospatial.org/is/18-010r7/18-010r7.html) or [PROJ definition (deprecated)](https://proj.org/usage/quickstart.html). Defaults to `4326` (EPSG code 4326) unless the client explicitly requests a different coordinate reference system.',
       'anyOf': [{'title': 'EPSG Code',
         'type': 'integer',
         'subtype': 'epsg-code',
         'minimum': 1000,
         'examples': [3857]},
        {'title': 'WKT2', 'type': 'string', 'subtype': 'wkt2-definition'},
        {'title': 'PROJ definition',
         'type': 'string',
         'subtype': 'proj-definition',
         'deprecated': True}],
       'default': 4326}},
     'title': 'Bounding Box'}]},
  {'name': 'temporal_extent',
   'description': 'Limits the data to load from the collection to the specified left-closed temporal interval. Applies to all temporal dimensions. The interval has to be specified as an array with exactly two elements:\n\n1. The first element is the start of the temporal interval. The specified instance in time is **included** in the interval.\n2. The second element is the end of the temporal interval. The specified instance in time is **excluded** from the interval.\n\nThe specified temporal strings follow [RFC 3339](https://www.rfc-editor.org/rfc/rfc3339.html). Also supports open intervals by setting one of the boundaries to `null`, but never both.\n\nSet this parameter to `null` to set no limit for 

In [8]:
options = {
    "doGeometry": True, 
    "doStitching": True,
    "doGeometryCheck": True, 
    "doToa": True, 
    "doInterCalibration": True, 
    "doAtmcor": True, 
    "doNbar": True, 
    "doSbaf": True
}

In [9]:
# SEN2LIKE
collection      = 'sentinel-2-l1c'
spatial_extent  = {"west": 15.99, "east": 16, "south": 46.99, "north": 47}
temporal_extent = ["2023-06-01", "2023-06-28"]

S2 = conn.load_collection(collection,spatial_extent=spatial_extent,temporal_extent=temporal_extent)
s2l = S2.process('sen2like', {
    'spatial_extent': spatial_extent, 
    'temporal_extent': temporal_extent, 
    'target_product': 'L2F', 
    'target_resolution': 10,
    'options': options})
s2l_save = s2l.save_result(format="NetCDF") # "GTiff"

job = s2l_save.create_job()
job.start_job()

In [13]:
job

<BatchJob job_id='5a08e174-026b-46ef-a086-9081c96359ed'>

In [14]:
job.job_id, job.status()

('5a08e174-026b-46ef-a086-9081c96359ed', 'running')

In [15]:
results = job.get_results()
metadata = results.get_metadata()

In [17]:
metadata

{'stac_version': '1.0.0',
 'stac_extensions': None,
 'type': 'Collection',
 'id': '56383c4a-3445-4838-8af6-891144a6418e',
 'title': None,
 'description': 'Auto generate stac collection for OpenEO Job results.',
 'keywords': None,
 'version': None,
 'deprecated': False,
 'license': 'proprietary',
 'providers': None,
 'extent': {'spatial': {'bbox': [[14.823254,
     46.742838,
     16.612428,
     47.965445]]},
  'temporal': {'interval': [['2023-06-12T16:36:48.998475Z',
     '2023-06-12T16:36:52.039844Z']]}},
 'links': [{'rel': 'self',
   'href': 'https://openeo-dev.eodc.eu/openeo/1.1.0/jobs/56383c4a-3445-4838-8af6-891144a6418e/results',
   'type': 'application/json',
   'title': None},
  {'rel': 'root',
   'href': 'https://openeo-dev.eodc.eu/openeo/1.1.0/jobs/56383c4a-3445-4838-8af6-891144a6418e/results',
   'type': 'application/json',
   'title': None},
  {'rel': 'canonical',
   'href': 'https://openeo-dev.eodc.eu/openeo/1.1.0/jobs/56383c4a-3445-4838-8af6-891144a6418e/results?Expires=1

### sen2like in openEO

- The Sen2Like, demonstration processor has been developed by ESA in the framework of the EU Copernicus programme (https://www.copernicus.eu/). The main goal of Sen2Like is to generate Sentinel-2 like harmonised/fused surface reflectances with higher periodicity by integrating additional compatible optical mission sensors.

- openEO provides an option to run the sen2like processor on the requested extent on an openEO backend. The backend takes care of getting the required input data and the computational resources to run sen2like on. openEO users can start sen2like jobs on the backend and download the results as in the example given above.

### Auxiliary data (openEO takes care of the datasets)

- sen2like expects Sentinel 2 and Landsat data
    - additionally uses CAMS data:
        - Total Aerosol Optical Depth at 550nm
        - Total column water vapour
        - Mean sea level pressure
        - GEMS Total column ozone
    - Sentinel 2 reference images:
        - provided by ESA: https://s2gri.csgroup.space/?.language=en#/?.language=en
        - e.g. https://s3.eu-central-2.wasabisys.com/s2-mpc/catalog/GRI_L1C/T04/T04Q/T04QCJ_item.json
        
### Results 

- openEO results (.nc or .tif) instead of .SAFE output format
    - openEO output formats:
        - GTiff, NetCDF (Rastercube)
    - sen2like .SAFE
        - AUX_DATA
        - GRANULE
        - DATASTRIP
        - HTML
        - MTD.xml
        
    - L2H, L2F
    
    - B01_60m, B11_20m, B02_10m, B12_20m, B03_10m,  B8A_20m, B04_10m